In [1]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

In [3]:
# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
json.loads(ret)

{'result': [{'url': 'https://www.walkerplus.com/event_list/ar0313/sc309880d/',
   'title': '東京駅(東京都)周辺のイベント - ウォーカープラス',
   'content': '開催中 2025年3月20日(祝)～6月3日(火) 京橋駅(東京都), 宝町駅(東京都), 日本橋駅(東京都), 銀座一丁目駅(東京都), 東京駅(東京都) CREATIVE MUSEUM TOKYO(クリエイティブ ミュージアム トウキョウ) *   [美術展・博物展](https://www.walkerplus.com/event_list/eg0107/) 開催中 2025年3月20日(祝)～6月3日(火) 京橋駅(東京都), 宝町駅(東京都), 日本橋駅(東京都), 銀座一丁目駅(東京都), 東京駅(東京都) CREATIVE MUSEUM TOKYO(クリエイティブ ミュージアム トウキョウ) *   [グルメ・フードフェス](https://www.walkerplus.com/event_list/eg0117/) *   ![Image 6](https://www.walkerplus.com/asset/images/common/ico_charge.svg) 終了間近 2025年4月1日(火)～5月31日(土) 東京駅(東京都), 二重橋前駅(東京都), 京橋駅(東京都), 大手町駅(東京都), 有楽町駅(東京都) *   [グルメ・フードフェス](https://www.walkerplus.com/event_list/eg0117/) *   ![Image 8](https://www.walkerplus.com/asset/images/common/ico_parking.svg) 2025年4月5日(土)～6月15日(日) 東京駅(東京都), 二重橋前駅(東京都), 大手町駅(東京都), 日本橋駅(東京都), 京橋駅(東京都) *   [美術展・博物展](https://www.walkerplus.com/event_list/eg0107/) 2025年3月1日(土)～6月1日(日) 京橋駅(東京都), 宝町駅(東京都), 日本橋駅(東京都), 東京駅(

In [12]:
# ツール定義
def define_tools():
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "指定した質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

In [28]:
# APIからのレスポンスをdictに変換する関数
def normalize_message(msg):
    """OpenAIのChatCompletionMessageをdictに変換"""
    return {
        "role": msg.role,
        "content": msg.content,
        "tool_calls": getattr(msg, "tool_calls", None)
    }

In [27]:
# 言語モデルへの質問を行う関数
def ask_question(messages, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    return response

In [25]:
# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, messages):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 既存の履歴に、assistant の tool_calls発話と tool からの応答を追加して再呼び出し
    # assistant メッセージを dict に正規化
    assistant_message = {
        "role": response.choices[0].message.role,
        "content": response.choices[0].message.content,
        "tool_calls": response.choices[0].message.tool_calls,
    }
    new_messages = messages + [
        assistant_message,
        {
            "tool_call_id": tool.id,
            "role": "tool",
            "content": function_response,
        },
    ]

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=new_messages
    )
    # 最終応答をdict化して返す
    return normalize_message(response_after_tool_call.choices[0].message)


In [26]:
# ユーザーからの質問を処理する関数
def process_response(messages, tools):
    if isinstance(messages, str):
        messages = [{"role": "user", "content": messages}]

    response = ask_question(messages, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        return handle_tool_call(response, messages)
    else:
        return normalize_message(response.choices[0].message)

In [20]:
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)

東京都と沖縄県の面積を比較すると、沖縄県の方が広いです。具体的には、最新のデータによると、沖縄県の面積は約2,276.6平方キロメートルであるのに対し、東京都の面積は約2,194平方キロメートルです。このため、沖縄県が東京都よりも広いと言えます。


In [21]:
tools = define_tools()

# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)

最近1ヶ月以内の東京駅に関連するイベント情報をいくつか紹介します。

1. **KITTEイベントカレンダー**
   - URL: [KITTEのイベントカレンダー](https://marunouchi.jp-kitte.jp/event/calendar.jsp)
   - 内容: 東京駅近くのKITTE内で様々なイベントが行われています。詳細は公式サイトを参照してください。

2. **東京都のイベント情報**
   - URL: [東京フェスティバル](https://www.tokyofes.info/)
   - 内容: 東京の様々な場所で行われるイベントやフェスティバルの情報が掲載されています。

3. **楽しめる東京のイベント一覧**
   - URL: [Enjoy Tokyo](https://www.enjoytokyo.jp/event/list/)
   - 内容: 東京で開催されているイベントの一覧を幅広く紹介しています。様々なジャンルのイベントをチェックできます。

4. **Walker+ - 東京のイベント情報**
   - URL: [Walker+ 東京 フェスティバル情報](https://www.walkerplus.com/top/ar0313/)
   - 内容: 東京でのイベント情報やおすすめのアクティビティに関する情報を提供しています。

5. **東京ドームイベントスケジュール**
   - URL: [東京ドームイベントスケジュール](https://www.tokyo-dome.co.jp/dome/event/schedule.html)
   - 内容: 東京ドームで開催されるコンサートやイベントのスケジュールが確認できます。

これらのウェブサイトを訪れることで、最新のイベント情報を得ることができますので、ぜひご確認ください。


In [29]:
# チャットボットへの組み込み
tools = define_tools()

messages=[]

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        del_message = messages.pop(0)

    # 言語モデルに質問
    response_message = process_response(messages, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message["content"], flush=True)
    messages.append(response_message)

print("\n---ご利用ありがとうございました！---")

'質問:東北6県について'

東北地方は、日本の本州北部に位置し、以下の6つの県から構成されています。

1. **青森県** - 県都は青森市で、新鮮な海産物やりんごが特産です。
2. **岩手県** - 県都は盛岡市で、自然豊かな地域が多く、文化遺産や温泉が点在します。
3. **宮城県** - 県都は仙台市で、東北の経済・文化の中心地として知られています。
4. **秋田県** - 県都は秋田市で、秋田美人や秋田犬が有名です。
5. **山形県** - 県都は山形市で、さくらんぼや米沢牛が特産です。
6. **福島県** - 県都は福島市で、観光地や温泉が数多くあります。

これらの県は、太平洋側に面した地域であり、独自の文化、自然、おいしい食べ物などが魅力とされています。冬は雪が多く、スキーや温泉が楽しめる場所としても人気があります。

詳しい情報は、以下のリンクからご覧いただけます：
- [東北地方 - Wikipedia](https://ja.wikipedia.org/wiki/%E6%9D%B1%E5%8C%97%E5%9C%B0%E6%96%B9) 
- [日本の観光メディアMATCHAの情報](https://matcha-jp.com/jp/1555) 
- [地域情報サイトROKKON](https://rokkkkkkon.jp/) 

興味のある具体的な事柄についてお知らせいただければ、さらに詳細な情報をご提供いたします。


'質問:山形県のおすすめのお土産を検索してほしい'

山形県のおすすめのお土産はいくつかあり、以下にいくつかの選択肢を紹介します。

1. **さくらんぼ**: 山形県はさくらんぼの産地として知られています。特に"佐藤錦"は非常に人気があります。

2. **山形の米**: 山形は米どころでもあり、特に「つや姫」と「はえぬき」が有名です。新鮮なお米を土産にできるのが魅力です。

3. **蔵王のチーズ**: 蔵王産のチーズは、豊かな風味とクリーミーさが特徴で、食べ応えがあります。

4. **山形の漬物**: しば漬けやらっきょう漬けなど、地元の伝統的な漬物も人気があります。

5. **米沢牛**: 高級和牛である米沢牛は、山形の特産品として有名です。冷凍肉をお土産に持って帰ることも可能です。

6. **山形名物のどぶろく**: 地元の酒蔵が作るどぶろくは、ユニークな風味が楽しめるお酒です。

詳しいお土産の情報やランキングについては、以下のリンクをご参照ください：

- [山形のお土産ランキングの記事](https://www.jalan.net/news/article/95650/)  
- [お土産のリスト](https://shonai-nandemoya.net/item-list?categoryId=36125)  
- [さらに多くのお土産の情報](https://sakidori.co/article/1319845)

これらの情報を参考に、山形県を訪れた際のお土産選びを楽しんでください！


'質問:イチオシはどれ？'

山形県のイチオシのお土産は以下の3つです。

1. **さくらんぼ**（特に佐藤錦）
   - 山形県は日本有数のさくらんぼの産地で、美味しい果物として人気があります。特に「佐藤錦」は甘さと酸味のバランスが良く、贈り物にも最適です。

2. **米沢牛**
   - 高級和牛として有名な米沢牛は、柔らかく、風味豊かな肉質が特徴です。お土産に冷凍肉を持ち帰ることもできますので、特別な贈り物にぴったりです。

3. **だだちゃ豆**
   - 山形の特産品である「だだちゃ豆」は、甘味が強く、鮮やかな緑色が特徴です。冷凍されて販売されることが多く、手軽に持ち帰ることができます。

これらのお土産は、山形の風土や文化を感じられるものばかりです。特にさくらんぼは季節限定のため、訪問時期に注意が必要ですが、旬の味を楽しむにはおすすめです。

詳しい情報は、以下のリンクを参考にしてください：
- [山形県のお土産まとめ](https://www.jalan.net/news/article/95650/) 
- [山形のお土産一覧](https://sakidori.co/article/1319845) 

ぜひ、山形の魅力を感じるお土産選びに役立ててください！

---ご利用ありがとうございました！---
